In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/LP_3_Weekly-Production-Metallurgical-Company/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 15:47:59.107238


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we need to define the parameters, decision variables, objective function, and constraints.

Parameters:
Let's define the following parameters:
- \( c_{ij} \): The cost of operation \( j \) for product \( i \), where \( i \in \{P1, P2, P3\} \) and \( j \in \{smelting, mechanisation, assembly\} \).
- \( s_{ij} \): The selling price of product \( i \).
- \( t_{ij} \): The time required for operation \( j \) for product \( i \).
- \( C_j \): The total available time for operation \( j \).

Decision Variables:
Let's define the following decision variables:
- \( x_{i} \): The number of units of product \( i \) to produce.
- \( y_{i} \): The number of units of product \( i \) to subcontract, where \( i \in \{P1, P2\} \). Note that for product P3, \( y_{P3} = 0 \) because it cannot be subcontracted.

Objective Function:
The objective is to maximize weekly profits, which can be expressed as the difference between the total revenue and the total costs. The total revenue is the sum of the revenue from each product, and the total costs are the sum of the costs from each operation for each product. The objective function can be written as:

\( \max \sum_{i} \left(s_{i} x_{i} - c_{i,smelting} x_{i} - c_{i,mechanisation} x_{i} - c_{i,assembly} x_{i} - c_{i,subcontract} y_{i}\right) \)

Constraints:
The constraints for this problem are the capacity constraints for each operation. The total time used for each operation cannot exceed the available time. The constraints can be written as:

- For smelting: \( t_{P1,smelting} (x_{P1} - y_{P1}) + t_{P2,smelting} (x_{P2} - y_{P2}) + t_{P3,smelting} x_{P3} \leq C_{smelting} \)
- For mechanisation: \( t_{P1,mechanisation} x_{P1} + t_{P2,mechanisation} x_{P2} + t_{P3,mechanisation} x_{P3} \leq C_{mechanisation} \)
- For assembly: \( t_{P1,assembly} x_{P1} + t_{P2,assembly} x_{P2} + t_{P3,assembly} x_{P3} \leq C_{assembly} \)

In addition, we have the non-negativity constraints: \( x_{i} \geq 0 \) and \( y_{i} \geq 0 \) for all \( i \).

This completes the formulation of the mathematical optimization model for the problem.

In [6]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we need to define the parameters, decision variables, objective function, and constraints.

Parameters:
Let's define the following parameters:
- \( c_{ij} \): The cost of operation \( j \) for product \( i \), where \( i \in \{P1, P2, P3\} \) and \( j \in \{smelting, mechanisation, assembly\} \).
- \( s_{ij} \): The selling price of product \( i \).
- \( t_{ij} \): The time required for operation \( j \) for product \( i \).
- \( C_j \): The total available time for operation \( j \).

Decision Variables:
Let's define the following decision variables:
- \( x_{i} \): The number of units of product \( i \) to produce.
- \( y_{i} \): The number of units of product \( i \) to subcontract, where \( i \in \{P1, P2\} \). Note that for product P3, \( y_{P3} = 0 \) because it cannot be subcontracted.

Objective Function:
The objective is to maximize weekly profits, which can be expressed as the difference between the total revenue

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters (using sample data)
model.c = {
    ('P1', 'smelting'): 10,
    ('P1', 'mechanisation'): 20,
    ('P1', 'assembly'): 30,
    ('P2', 'smelting'): 15,
    ('P2', 'mechanisation'): 30,
    ('P2', 'assembly'): 45,
    ('P3', 'smelting'): 20,
    ('P3', 'mechanisation'): 40,
    ('P3', 'assembly'): 60
}

model.s = {'P1': 100, 'P2': 150, 'P3': 200}

model.t = {
    ('P1', 'smelting'): 1,
    ('P1', 'mechanisation'): 2,
    ('P1', 'assembly'): 3,
    ('P2', 'smelting'): 1.5,
    ('P2', 'mechanisation'): 3,
    ('P2', 'assembly'): 4.5,
    ('P3', 'smelting'): 2,
    ('P3', 'mechanisation'): 4,
    ('P3', 'assembly'): 6
}

model.C = {'smelting': 100, 'mechanisation': 200, 'assembly': 300}

# Define the decision variables
model.x = Var(range(1, 4), domain=NonNegativeReals)
model.y = Var(range(1, 3), domain=NonNegativeReals)

# Define the objective function
def obj_rule(model):
    return sum((model.s[i] * model.x[i] - model.c[i, 'smelting'] * model.x[i] - model.c[i, 'mechanisation'] * model.x[i] - model.c[i, 'assembly'] * model.x[i] - model.c[i, 'subcontract'] * model.y[i]) for i in range(1, 4))
model.obj = Objective(rule=obj_rule, sense=maximize)

# Define the constraints
def smelting_rule(model):
    return sum(model.t[i, 'smelting'] * (model.x[i] - model.y[i if i < 3 else 2]) for i in range(1, 4)) <= model.C['smelting']
model.smelting_ constraint = Constraint(rule=smelting_rule)

def mechanisation_rule(model):
    return sum(model.t[i, 'mechanisation'] * model.x[i] for i in range(1, 4)) <= model.C['mechanisation']
model.mechanisation_constraint = Constraint(rule=mechanisation_rule)

def assembly_rule(model):
    return sum(model.t[i, 'assembly'] * model.x[i] for i in range(1, 4)) <= model.C['assembly']
model.assembly_constraint = Constraint(rule=assembly_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in range(1, 4):
    print(f"x{i}: {model.x[i].value}, y{i}: {model.y[i].value if i < 3 else 0}")
print(f"Objective value: {model.obj()}")
```

Note that this code uses sample data for the parameters \( c_{ij} \), \( s_{i} \), \( t_{ij} \), and \( C_{j} \). You can replace this sample data with the actual data for your specific problem. The code also assumes that the products are labeled as `P1`, `P2`, and `P3` and the operations are labeled as `smelting`, `mechanisation`, and `assembly`. If your problem uses different labels, you will need to modify the code accordingly.

In [10]:
print(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters (using sample data)
model.c = {
    ('P1', 'smelting'): 10,
    ('P1', 'mechanisation'): 20,
    ('P1', 'assembly'): 30,
    ('P2', 'smelting'): 15,
    ('P2', 'mechanisation'): 30,
    ('P2', 'assembly'): 45,
    ('P3', 'smelting'): 20,
    ('P3', 'mechanisation'): 40,
    ('P3', 'assembly'): 60
}

model.s = {'P1': 100, 'P2': 150, 'P3': 200}

model.t = {
    ('P1', 'smelting'): 1,
    ('P1', 'mechanisation'): 2,
    ('P1', 'assembly'): 3,
    ('P2', 'smelting'): 1.5,
    ('P2', 'mechanisation'): 3,
    ('P2', 'assembly'): 4.5,
    ('P3', 'smelting'): 2,
    ('P3', 'mechanisation'): 4,
    ('P3', 'assembly'): 6
}

model.C = {'smelting': 100, 'mechanisation': 200, 'assembly': 300}

# Define the decision variables
model.x = Var(range(1, 4), domain=NonNegativeReals)
model.y = Var(range(1, 3), domain=NonNegativeReals)

# Defin

### Code Executability

In [11]:
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the parameters (using sample data)
model.c = {
    ('P1', 'smelting'): 10,
    ('P1', 'mechanisation'): 20,
    ('P1', 'assembly'): 30,
    ('P2', 'smelting'): 15,
    ('P2', 'mechanisation'): 30,
    ('P2', 'assembly'): 45,
    ('P3', 'smelting'): 20,
    ('P3', 'mechanisation'): 40,
    ('P3', 'assembly'): 60
}

model.s = {'P1': 100, 'P2': 150, 'P3': 200}

model.t = {
    ('P1', 'smelting'): 1,
    ('P1', 'mechanisation'): 2,
    ('P1', 'assembly'): 3,
    ('P2', 'smelting'): 1.5,
    ('P2', 'mechanisation'): 3,
    ('P2', 'assembly'): 4.5,
    ('P3', 'smelting'): 2,
    ('P3', 'mechanisation'): 4,
    ('P3', 'assembly'): 6
}

model.C = {'smelting': 100, 'mechanisation': 200, 'assembly': 300}

# Define the decision variables
model.x = Var(range(1, 4), domain=NonNegativeReals)
model.y = Var(range(1, 3), domain=NonNegativeReals)

# Define the objective function
def obj_rule(model):
    return sum((model.s[i] * model.x[i] - model.c[i, 'smelting'] * model.x[i] - model.c[i, 'mechanisation'] * model.x[i] - model.c[i, 'assembly'] * model.x[i] - model.c[i, 'subcontract'] * model.y[i]) for i in range(1, 4))
model.obj = Objective(rule=obj_rule, sense=maximize)

# Define the constraints
def smelting_rule(model):
    return sum(model.t[i, 'smelting'] * (model.x[i] - model.y[i if i < 3 else 2]) for i in range(1, 4)) <= model.C['smelting']
model.smelting_ constraint = Constraint(rule=smelting_rule)

def mechanisation_rule(model):
    return sum(model.t[i, 'mechanisation'] * model.x[i] for i in range(1, 4)) <= model.C['mechanisation']
model.mechanisation_constraint = Constraint(rule=mechanisation_rule)

def assembly_rule(model):
    return sum(model.t[i, 'assembly'] * model.x[i] for i in range(1, 4)) <= model.C['assembly']
model.assembly_constraint = Constraint(rule=assembly_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in range(1, 4):
    print(f"x{i}: {model.x[i].value}, y{i}: {model.y[i].value if i < 3 else 0}")
print(f"Objective value: {model.obj()}")

SyntaxError: invalid syntax (3146439518.py, line 47)

### Solution Correctness

In [17]:
from pyomo.environ import *


# Define sets
products = ["P1", "P2", "P3"]
subcontractable_products = ["P1", "P2"]

# Define parameters
smelting_time = {"P1": 6, "P2": 10, "P3": 8}  
mechanization_time = {"P1": 6, "P2": 3, "P3": 8} 
assembly_time = {"P1": 3, "P2": 2, "P3": 2}  

# Cost parameters
smelting_cost = {"P1": 0.30, "P2": 0.50, "P3": 0.40}
subcontracted_smelting_cost = {"P1": 0.50, "P2": 0.60, "P3" : 0}
mechanization_cost = {"P1": 0.20, "P2": 0.10, "P3": 0.27}
assembly_cost = {"P1": 0.30, "P2": 0.20, "P3": 0.20}

# Selling price
selling_price = {"P1": 1.50, "P2": 1.80, "P3": 1.97}

# Available production time
available_smelting_time = 8000
available_mechanization_time = 12000
available_assembly_time = 10000

model = ConcreteModel()

# Define the decision variables
model.x = Var(products, domain=NonNegativeReals)
model.y = Var(products, domain=NonNegativeReals)

model.y3_constraint = Constraint(expr=model.y['P3'] == 0)

# Define the objective function
def obj_rule(model):
    return sum((selling_price[p] * model.x[p] - smelting_cost[p] * model.x[p] - mechanization_cost[p] * model.x[p] - assembly_cost[p] * model.x[p] - subcontracted_smelting_cost[p] * model.y[p]) for p in products)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Define the constraints
def smelting_rule(model):
    return sum(smelting_time[p] * (model.x[p] - model.y[p]) for p in products) <= available_smelting_time
model.smelting_constraint = Constraint(rule=smelting_rule)

def mechanisation_rule(model):
    return sum(mechanization_time[p] * model.x[p] for p in products) <= available_mechanization_time
model.mechanisation_constraint = Constraint(rule=mechanisation_rule)

def assembly_rule(model):
    return sum(assembly_time[p] * model.x[p] for p in products) <= available_assembly_time
model.assembly_constraint = Constraint(rule=assembly_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal Solution:")
for p in products:
    print(f"x{p}: {model.x[p].value}, y{p}: {model.y[p].value}")
print(f"Objective value: {model.obj()}")

Optimal Solution:
xP1: 0.0, yP1: 0.0
xP2: 4000.0, yP2: 3200.0
xP3: 0.0, yP3: 0.0
Objective value: 2080.0
